# Importing Libraries

In [1]:
import feedparser as fp
import json
import newspaper
from newspaper import Article
from time import mktime
from datetime import datetime
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')
import pandas as pd
from pandas.io.json import json_normalize
import nltk
import re
from rouge import Rouge 
from collections import Counter
from geotext import GeoText
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.downloader.download('maxent_ne_chunker')
nltk.downloader.download('words')
nltk.downloader.download('treebank')
nltk.downloader.download('maxent_treebank_pos_tagger')
import string
from gensim.summarization.summarizer import summarize

from sumy.parsers.plaintext import PlaintextParser #We're choosing a plaintext parser
from sumy.nlp.tokenizers import Tokenizer 
from sumy.summarizers.lex_rank import LexRankSummarizer #We're choosing Lexrank, other algorithms are also built in
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.luhn import LuhnSummarizer

import articleDateExtractor


global str

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Soumi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Soumi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Soumi\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Soumi\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\Soumi\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     C:\Users\Soumi\AppData\Roaming\nltk_data...
[nltk_data]   Package maxen

In [2]:
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize

# Classifier 

In [3]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas as pd, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
import spacy
import re
import csv
from sklearn.naive_bayes import MultinomialNB
from  sklearn.metrics  import accuracy_score

Using TensorFlow backend.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Soumi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Data Cleaning

In [4]:
lemma = nltk.WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
def preprocess(rawText):
    cleanText = re.sub(r'[^\w\s]','',rawText.lower())
    tokens = word_tokenize(cleanText)
    tokens = [token for token in tokens if not token in stop_words]
    tokens = [lemma.lemmatize(token) for token in tokens ]
    processedText = (" ".join(tokens))
    return processedText

# Importing dataset

In [5]:
data = open(r'C:\Users\soumi\indiaCorpus_01_04.csv')

target, notes = [], []
reader = csv.reader(data, delimiter=",")
next(reader, None)
for i, line in enumerate(reader):
    target.append(line[1])
    rawText = line[0]
    notes.append(preprocess(rawText))

dataSet = pd.DataFrame()
dataSet['notes'] = notes
dataSet['target'] = target
dataSet = dataSet.sample(frac=1).reset_index(drop=True)

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(dataSet['notes'], dataSet['target'])

encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

vector = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
vector.fit(dataSet['notes'])
train_x =  vector.transform(train_x)
valid_x =  vector.transform(valid_x)

# Naieve Bayes Event Classifier

In [6]:
clf = MultinomialNB().fit(train_x, train_y)
predicted = clf.predict(valid_x)
print(accuracy_score(valid_y,predicted))

0.8559485530546623


# SVM Event Classifier

In [7]:
from sklearn import svm
SVM = svm.LinearSVC()
SVM.fit(train_x, train_y)
predict = SVM.predict(valid_x)
accuracy_score(valid_y,predict)

0.9459807073954984

# Random Forest Event Classifier

In [8]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=10)
rf.fit(train_x, train_y)
predict = rf.predict(valid_x)
accuracy_score(valid_y,predict)

0.912540192926045

# Actor Classifier

In [9]:
data = open(r'C:\Users\soumi\indiaActorsCorpus2_2010_2019.csv')

target, notes = [], []
reader = csv.reader(data, delimiter=",")
next(reader, None)
for i, line in enumerate(reader):
    target.append(line[1])
    rawText = line[0]
    notes.append(preprocess(rawText))

df = pd.DataFrame()
df['notes'] = notes
df['target'] = target
df['target'] = df['target'].replace({'CPI (Maoist): Communist Party of India (Maoist)':'CPI'})
df['target'] = df['target'].replace({'BJP: Bharatiya Janata Party':'BJP'})
df['target'] = df['target'].replace({'INC: Indian National Congress':'INC'})
df['target'] = df['target'].replace({'Protesters':'Protesters'})
df['target'] = df['target'].replace({'Rioters':'Rioters'})

df = df.sample(frac=1).reset_index(drop=True)

actrain_x, acvalid_x, actrain_y, acvalid_y = model_selection.train_test_split(df['notes'], df['target'])

acencoder = preprocessing.LabelEncoder()
actrain_y = acencoder.fit_transform(actrain_y)
acvalid_y = acencoder.fit_transform(acvalid_y)

acvector = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=3000)
acvector.fit(df['notes'])
actrain_x =  acvector.transform(actrain_x)
acvalid_x =  acvector.transform(acvalid_x)



# Random Forest Actor Classifier

In [10]:
from sklearn.ensemble import RandomForestClassifier
actorRf = RandomForestClassifier(n_estimators=50)
actorRf.fit(actrain_x, actrain_y)
acpredict = actorRf.predict(acvalid_x)
accuracy_score(acvalid_y,acpredict)

0.019293860334322763

# SVM Actor Classifier

In [11]:
from sklearn import svm
acSVM = svm.LinearSVC()
acSVM.fit(actrain_x, actrain_y)
predict = acSVM.predict(acvalid_x)
accuracy_score(acvalid_y,predict)

0.014919543821277925

In [12]:
LIMIT = 4
data = {}
data['newspapers'] = {}
articlesData ={}
collections = {
    'total' : []
}


In [13]:
# NewspaperLinks.json contains RSS feeds and links

with open('newspaperLinks.json') as file:
    sources = json.load(file)
sources

{'TOI': {'rss': 'https://timesofindia.indiatimes.com/rssfeeds/-2128936835.cms',
  'link': 'https://timesofindia.indiatimes.com/elections/news'},
 'Hindustan Times': {'rss': 'https://www.hindustantimes.com/rss/india/rssfeed.xml',
  'link': 'https://www.hindustantimes.com/'},
 'India Today': {'rss': 'https://www.indiatoday.in/rss/1206578',
  'link': 'https://indianexpress.com/?s=politics'},
 'Business Standard': {'rss': 'https://www.business-standard.com/rss/latest.rss',
  'link': 'https://www.business-standard.com/lok-sabha-elections-2019'}}

# Summarization using lexRank, Location using GeoText

In [14]:
def summarizeText(text):
    string=text
    parser = PlaintextParser.from_string(string, Tokenizer("english"))

    summarizer = LexRankSummarizer()
    summary_lex = summarizer(parser.document, 1) #Summarize the document with 5 sentences
    summary_1=''
    for line in summary_lex:
        summary_1+=str(line)
    return summary_1

def getLocation(ents, text):
    loc = ""
    items = [x.text for x in ents if x.label_ in 'LOC']
    loc_ = [Counter(items).most_common(1)]
    places = GeoText(text)
    topPlace = [Counter(places.cities).most_common(1)]
    
    places = (places.cities)
    locList = [i for i in places if i in items]
    total = items+locList 
    location_ = [Counter(total).most_common(1)]
    if len(location_) == 0:
        if(len(loc_))!=0:
            loc = loc_[0][0][0]
        elif (len(topPlace)!=0):
            loc = topPlace[0]
        else:
            loc = ""
    else:
        if(len(location_))!=0:
            if(len(location_[0]))!=0:
                if(len(location_[0][0]))!=0:
                    loc = location_[0][0][0]
        else:
            loc = "unknown"
    return loc

# def getEventType(tokens):
# #     eventTypes ={
# #     'civilianViolence':['rebel','violence','militia','war','rape','torture','kill','shoot','fight','murder','molest','attack','terror','battle','assault','blood'],
# #     'riotsProtest':['protest','riot']
# #     }
#     eventType = "other"
#     for event, types in eventTypes.items():
#         tag = [token for token in tokens if any(typ in token for typ in types)]
#         if len(tag) != 0:
#             eventType = event
#             break
#     return eventType

# News Scraper and Classifier

In [15]:
collections = {
    'total' : []
}
from time import mktime
from datetime import datetime
from lexrank import STOPWORDS, LexRank

event_type = ""
data = {}
data['newspapers'] = {}
articlesData ={}
collectedNews = {}
events = ['arrest','arrested','violence against civilians','rioting','protesting','riots','protest','battles','violence','remote violence','assault','attack','bloodshed','brutality','clash','confusion','cruelty','disorder','disturbance','fighting','rampage','struggle','terrorism','abandon','acuteness','bestiality','blowup','coercion','compulsion','constraint','destructiveness','duress','ferocity','fervor','fierceness','flap','frenzy','fury','fuss','harshness','murderousness','onslaught','passion','power','roughness','ruckus','rumble','savagery','scam','severity','sharpness','storm','storminess','tumult','turbulence','uproar','vehemence','wildness','brute','force','foul','play','raging','anarchy','brawl','disturbance','lawlessness','protest','storm','strife','trouble','turbulence','turmoil','uproar','anarchism','burst','commotion','confusion','distemper','flap','fray','free-for-all','fuss','hassle','misrule','mix-up','quarrel','racket','row','ruckus','ruction','rumble','rumpus','run-in','scene','shivaree','shower','snarl','stir','to-do','tumult','brannigan','mob','violence','street','fighting','wingding','action','assault','attack','bloodshed','bombing','campaign','clash','combat','conflict','crusade','encounter','fighting','hostility','skirmish','strife','struggle','war','warfare','barrage','brush','carnage','contention','engagement','fray','havoc','onset','onslaught','press','ravage','scrimmage','sortie','blitzkreig']
politics= ["politicize", "political affairs", "administration", "gandhi", "Modi", "politics", "politicians", "elections", "votes", "voters", "BJP", "Congress", "Supreme Court","Nationalism","polls","poll","voting"]
violence = ["violence", "criminal violence", "criminal", "crime", "criminal party", "election violence", "electoral violence","mob killing", "vigilante", "riots", "killing" ]
parties = ['bharatiya janata party', 'bjp', 'b.j.p.', 'bhaujan samaj party', 'bsp', 'b.s.p.', 'indian national congress', 'inc', 'i.n.c.', 'congress', 'nationalist congress party', 'ncp', 'n.c.p.','communist party of india','cpi','c.p.i.','aam aadmi party','aap','a.a.p.','samajwadi party','shiv sena','shs','rashtriya janta dal','rjd','r.j.d.']
    
eventTypes ={
'riots':['riot'],
'protests':['protest','march'], 
'civilianViolence':['rebel','violence','militia','war','rape','torture','kill','shoot','fight','murder','molest','attack','terror','battle','assault','blood']
}



count = 1

# Scrapping news articles from RSS news feeds and Indian news links
# If loop scrapes rss feeds and else part scrapes links

for source, link in sources.items():
    print(source)
    count = 1
    if 'rss' in link:
        parsedFeed = fp.parse(link['rss'])
        collected = {
            'rss': link['rss'],
            'link': link['link'],
            'article': []
        }
        for feed in parsedFeed.entries:
            
            try:

                articlesData ={}
                articles = Article(feed['links'][0]['href'])
                articles.download()
                articles.parse()
                articles.nlp()
                strg = re.sub(r'[^\w\s]','',articles.text)
                articleTokens = nlp(strg)

                tokens =[]
                for token in articleTokens:
                    tokens.append(token.text.lower())

                txt = str(articles.text).split()
#                 flag = True
#                 for i in txt:
#                     for j in events:
#                         if i==j:
#                             event_type = j
#                             flag = False
#                             break
#                         else:
#                             event_type = "other"
#                     if not flag:
#                         break
                keyWords = articles.keywords

                eventType = "other"
                if(([a for a in keyWords if any(b.lower() in a.lower() for b in politics)])):
                    print(articles.title)
                    articlesData['title'] = articles.title

                    article = nlp(articles.text)
                    ###Summary
                    sentences = [x for x in article.sents]
                    sentences = list( map(str, sentences) )
                    lxr = LexRank(articles.text)
                    lxr.get_summary(sentences)
                    summary = lxr.get_summary(sentences, summary_size=1, threshold=.1)
                    summary = (" ".join(summary))
                    
                    rawSummary = [preprocess(summary)]
                    textVector = vector.transform(rawSummary)
                    prediction = rf.predict(textVector)
                    print(encoder.inverse_transform(prediction))
                    articlesData['event'] = encoder.inverse_transform(prediction)[0]

                    dates = articleDateExtractor.extractArticlePublishedDate(feed['links'][0]['href'])
        #            print(dates)
                    if dates is not None:
                        localFormat = dates.strftime("%m/%d/%Y")
                        summaryDate = dates.strftime("On %d %b,")
                    elif feed.published_parsed is not None:
                        timestamp = (datetime.fromtimestamp(mktime(feed.published_parsed)).isoformat())
                        localFormat = datetime.strptime(timestamp, '%Y-%m-%dT%H:%M:%S').strftime('%m/%d/%Y')
                        summaryDate = datetime.strptime(timestamp, '%Y-%m-%dT%H:%M:%S').strftime('On %d %b,')
                    else:
                        summaryDate=""
                        localFormat=""
                    articlesData['eventDate'] = localFormat
                    articlesData['location'] = getLocation(articleTokens.ents, articles.text)
                    orgs = set(keyWords).intersection(set(parties))

                    if len(orgs) == 0:
                        if(articlesData['event']=="Riots"):
                            taggedOrg = "Rioters"
                        elif(articlesData['event']=="Protests"):
                            taggedOrg = "Protestors"
                        else:
                            taggedOrg = "other"
                    else:
                        taggedOrg=""
                        for org in orgs:
                            taggedOrg+=org+", "
                        taggedOrg=taggedOrg[:-2]

                    articlesData['partiesInvolved'] = taggedOrg
                    articlesData['source'] = source
                    totalSummary = []
                    totalSummary.append(summaryDate)
                    totalSummary.append(summary)
                    extractiveSummary  = (" ".join(totalSummary))
                    articlesData['summary'] = extractiveSummary
                    articlesData['link'] = (feed['links'][0]['href'])
                    collected['article'].append(articlesData)
#                 else:
#                     break
            except Exception as ex:
                print(ex)
        collections['total'].append(collected)
    else:
        parsedFeed = newspaper.build(link['link'], memoize_articles = False)
        collected = {
            'link': link['link'],
            'article': []
        }
        for feed in parsedFeed.articles:
#             if count > 1000:
#                 break
            try:
                feed.download()
                feed.parse()
                feed.nlp()
                if feed.publish_date is not None:
                    articlesData ={}
                    
                    strg = re.sub(r'[^\w\s]','',feed.text)
                    articleTokens = nlp(strg)
                    tokens =[]
                    for token in articleTokens:
                        tokens.append(token.text.lower())

                        txt = str(feed.text).split()
#                         for i in txt:
#                             for j in events:
#                                 if i==j:
#                                     event_type = j
#                                     break
#                                 else:
#                                     event_type = "other"
                    keyWords = feed.keywords
                    if(([a for a in keyWords if any(b.lower() in a.lower() for b in politics)])):
                        
                        articlesData['title'] = feed.title
                        article = nlp(feed.text)
                        ###Summary
                        sentences = [x for x in article.sents]
                        sentences = list( map(str, sentences) )
                        lxr = LexRank(feed.text)
                        lxr.get_summary(sentences)
                        summary = lxr.get_summary(sentences, summary_size=1, threshold=.1)
                        summary = (" ".join(summary))

                        rawSummary = [preprocess(summary)]
                        textVector = vector.transform(rawSummary)
                        prediction = rf.predict(textVector)
                        print(encoder.inverse_transform(prediction))
                        articlesData['event'] = encoder.inverse_transform(prediction)[0]
                        
#                         articlesData['event'] = getEventType(tokens)
                        articlesData['eventDate'] = feed.publish_date.strftime('%m/%d/%Y')
                        articlesData['location'] = getLocation(articleTokens.ents,feed.text)
                        
                        orgs = set(keyWords).intersection(set(parties))
                        if len(orgs) == 0:
                            if(articlesData['event']=="Riots"):
                                taggedOrg = "Rioters"
                            elif(articlesData['event']=="Protests"):
                                taggedOrg = "Protestors"
                            else:
                                taggedOrg = "other"
                        else:
                            taggedOrg=""
                            for org in orgs:
                                taggedOrg+=org+", "
                            taggedOrg=taggedOrg[:-2]

                        articlesData['partiesInvolved'] = taggedOrg
                        articlesData['source'] = source
                        totalSummary = []
                        totalSummary.append(feed.publish_date.strftime("On %d %b,"))
                        totalSummary.append(summary)
                        extractiveSummary  = (" ".join(totalSummary))
                        articlesData['summary'] = extractiveSummary
                        articlesData['link'] = link
                        collected['article'].append(articlesData)
#                     else:
#                         break
#                         count +=1
            except Exception as ex:
                print(ex)
        collections['total'].append(collected)

TOI
Decision on oil purchase after polls, India tells Iran
['Protests']
Extracting date from https://timesofindia.indiatimes.com/india/decision-on-oil-purchase-after-polls-india-tells-iran/articleshow/69333497.cms
Priyanka Gandhi offers best wishes to MP group shouting 'Modi, Modi'
['Protests']
Extracting date from https://timesofindia.indiatimes.com/india/priyanka-gandhi-offers-best-wishes-to-mp-group-shouting-modi-modi/articleshow/69331833.cms
West Bengal CM Mamata Banerjee be barred from campaigning: BJP to EC
['Protests']
Extracting date from https://timesofindia.indiatimes.com/india/west-bengal-cm-mamata-banerjee-be-barred-from-campaigning-bjp-to-ec/articleshow/69331796.cms
Ahead of Rahul’s Alwar visit, Congress says rape victims denied justice in BJP-ruled UP, J&K
['Protests']
Extracting date from https://timesofindia.indiatimes.com/india/ahead-of-rahuls-alwar-visit-congress-says-rape-victims-denied-justice-in-bjp-ruled-up-jk/articleshow/69330777.cms
'Outsiders' brought by Amit S

BJP demands CBI probe into Alwar gangrape case
['Protests']
Extracting date from https://www.indiatoday.in/india/story/bjp-demands-cbi-probe-into-alwar-gang-rape-case-1521404-2019-05-09?utm_source=rss
Alwar gangrape case: All six accused arrested, BJP holds protests
['Protests']
Extracting date from https://www.indiatoday.in/india/story/alwar-gangrape-case-nsix-accused-arrested-bjp-protests-1521325-2019-05-09?utm_source=rss
BJP respects Rajiv Gandhi, but has right to question his govt: Nirmala Sitharaman
['Protests']
Extracting date from https://www.indiatoday.in/india/story/bjp-respects-rajiv-gandhi-but-has-right-to-question-his-govt-nirmala-sitharaman-1521052-2019-05-09?utm_source=rss
Rajiv Gandhi assassination case: SC dismisses pleas opposing Tamil Nadu govt's move to release convicts
['Protests']
Extracting date from https://www.indiatoday.in/india/story/rajiv-gandhi-assassination-case-sc-dismisses-pleas-opposing-tamil-nadu-govt-move-to-release-convicts-1520799-2019-05-09?utm_sour

In [16]:
collections["total"]

[{'rss': 'https://timesofindia.indiatimes.com/rssfeeds/-2128936835.cms',
  'link': 'https://timesofindia.indiatimes.com/elections/news',
  'article': [{'title': 'Decision on oil purchase after polls, India tells Iran',
    'event': 'Protests',
    'eventDate': '05/15/2019',
    'location': 'Gulf',
    'partiesInvolved': 'Protestors',
    'source': 'TOI',
    'summary': 'On 15 May, He mentioned that 60 days timeline has been given to EU-3 and other parties to JCPOA for restoring the oil and banking channels,” said an official source.',
    'link': 'https://timesofindia.indiatimes.com/india/decision-on-oil-purchase-after-polls-india-tells-iran/articleshow/69333497.cms'},
   {'title': "Priyanka Gandhi offers best wishes to MP group shouting 'Modi, Modi'",
    'event': 'Protests',
    'eventDate': '05/14/2019',
    'location': '',
    'partiesInvolved': 'congress',
    'source': 'TOI',
    'summary': 'On 14 May, "Priyanka\'s impromptu gesture shows the Congress believes in compassion and l

In [17]:
frame = json_normalize(collections['total'])
rows = frame.shape[0]
frame

,article,link,rss
0,[{'title': 'Decision on oil purchase after pol...,https://timesofindia.indiatimes.com/elections/...,https://timesofindia.indiatimes.com/rssfeeds/-...
1,"[{'title': 'Wasn’t I prophetic, asks Aiyar in ...",https://www.hindustantimes.com/,https://www.hindustantimes.com/rss/india/rssfe...
2,"[{'title': 'Monserratte rape case: BJP, Cong s...",https://indianexpress.com/?s=politics,https://www.indiatoday.in/rss/1206578
3,[{'title': 'BJP goes to EC after 'attack' on A...,https://www.business-standard.com/lok-sabha-el...,https://www.business-standard.com/rss/latest.rss


In [18]:
totalDF = pd.DataFrame()
for row in range(0,rows):
    tempDF = pd.DataFrame()
    tempDF = json_normalize(collections['total'][row]['article'])
    totalDF = totalDF.append(tempDF, ignore_index = True) 


# Dataframe that contains scraped news which has been classified and summarized

In [19]:
totalDF = totalDF[totalDF['location']!= ""]

In [20]:
totalDF

,event,eventDate,link,location,partiesInvolved,source,summary,title
0,Protests,05/15/2019,https://timesofindia.indiatimes.com/india/deci...,Gulf,Protestors,TOI,"On 15 May, He mentioned that 60 days timeline ...","Decision on oil purchase after polls, India te..."
4,Protests,05/14/2019,https://timesofindia.indiatimes.com/india/outs...,Esplanade,bjp,TOI,"On 14 May, The Trinamool Congress alleged that...",'Outsiders' brought by Amit Shah for Kolkata r...
6,Protests,05/14/2019,https://timesofindia.indiatimes.com/india/sc-t...,Mau district,Protestors,TOI,"On 14 May, Track live election results , the b...",SC to hear on May 17 plea by SP-BSP alliance c...
10,Protests,05/14/2019,https://www.hindustantimes.com/lok-sabha-elect...,West Bengal,"congress, bjp",Hindustan Times,"On 14 May, The Left wants to support an altern...",Lok Sabha elections 2019: Left ready to back n...
13,Protests,05/14/2019,https://www.hindustantimes.com/lok-sabha-elect...,the Varanasi Lok Sabha,bjp,Hindustan Times,"On 14 May, The supporters of Akhilesh Yadav, w...",Lok Sabha elections 2019: Who will win the bat...
14,Protests,05/15/2019,https://www.indiatoday.in/india/story/atanasio...,South Goa,"congress, bjp",India Today,"On 15 May, Speaking to reporters Tuesday, Goa ...","Monserratte rape case: BJP, Cong spar over wom..."
16,Protests,05/14/2019,https://www.indiatoday.in/india/story/two-kara...,Pakistans Karachi,Protestors,India Today,"On 14 May, When we went to the PMO, the people...","2 Pakistani women, living in India for 24 year..."
17,Protests,05/14/2019,https://www.indiatoday.in/india/story/woman-ab...,Twitter,Protestors,India Today,"On 14 May, Asserting the 'rightful place' of D...","Video of woman abusing Dalits, praising PM Mod..."
20,Protests,05/13/2019,https://www.indiatoday.in/india/story/report-o...,Agras,Protestors,India Today,"On 13 May, With pollution concerns over the Ta...",Report on pollution to decide future of indust...
22,Protests,05/11/2019,https://www.indiatoday.in/india/story/pm-modi-...,Rajat Sharmas,Protestors,India Today,"On 11 May, ""I do not know whether this episode...",PM Modi curtails tenure of senior railway offi...
